<a href="https://colab.research.google.com/github/chia-yu0225/work/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 猜數字小遊戲

本專題透過 LINE Bot 與 Python 的整合，實作一個互動式「猜數字」遊戲。使用者只要傳訊息給 LINE 機器人，就可以開始遊戲、猜數字、獲得提示，並最終猜中答案。整個程式在 Google Colab 上執行，結合了雲端運算與即時互動，適合用來展示聊天機器人與 Web 技術的結合應用。

# 補充

| 類別            | 技術內容                | 說明                                                 |
| ------------- | ------------------- | -------------------------------------------------- |
| 使用者介面     | LINE Messaging API  | 使用者透過 LINE 傳送訊息互動                                  |
| 伺服器應用     | Flask + ngrok       | 利用 Flask 建立 Web 伺服器，透過 ngrok 將本地伺服器公開到網路上供 LINE 訪問 |
| 雲端開發      | Google Colab        | 全程在 Google Colab 上開發與執行，不需本地安裝環境                   |
| 使用者狀態管理   | Python 字典           | 以 user ID 作為 key，儲存每個使用者的答案、猜測次數與遊戲狀態              |
| 隨機性邏輯     | random 模組           | 使用 `random.randint(1, 100)` 生成隨機答案                 |
| 條件判斷與文字回應 | if/elif/else        | 分析使用者輸入並給出不同的回應（如「太小」、「太大」、「猜對了」等）                 |
| 資安與部署     | 使用 `userdata.get()` | 金鑰與敏感資訊不寫死在程式碼中，提升安全性                              |


# 遊戲流程

使用者輸入「開始遊戲』，系統隨機選出一個 1～100 的整數，並初始化猜測次數為 0，使用者開始猜數字，每輸入一個數字，系統會判斷是否大於、小於或等於答案，並記錄猜測次數，猜中答案
，顯示成功訊息，並告知共猜了幾次
，重新開始
，輸入「開始遊戲」可以重新開始新的一輪。

# 安全性設定

In [ ]:
from google.colab import userdata
userdata.set('LINE_CHANNEL_SECRET', '89bcaecc9df0b2e66a7447dd3b919a7c')
userdata.set('LINE_CHANNEL_ACCESS_TOKEN', '在這裡填入你的 Access Token')
userdata.set('ngrok_token', '在這裡填入你的 ngrok authtoken')

# 安裝必要套件

In [ ]:
!pip install flask-ngrok line-bot-sdk --quiet

# 程式主體

In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage,
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent
import random

app = Flask(__name__)
run_with_ngrok(app)
ngrok.set_auth_token(userdata.get('ngrok_token'))

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))


game_states = {}

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_id = event.source.user_id
    user_msg = event.message.text.strip()

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)


        if user_msg == "開始遊戲":
            answer = random.randint(1, 100)
            game_states[user_id] = {
                "answer": answer,
                "active": True,
                "attempts": 0
            }
            reply = "🎲 猜數字遊戲開始！請輸入 1 到 100 的數字。"

        elif user_id not in game_states or not game_states[user_id]["active"]:
            reply = "請先輸入『開始遊戲』來開始新一輪猜數字！"

        else:
            try:
                guess = int(user_msg)
                state = game_states[user_id]
                state["attempts"] += 1

                if guess < state["answer"]:
                    reply = "太小了，再試一次！"
                elif guess > state["answer"]:
                    reply = "太大了，再試一次！"
                else:
                    reply = (
                        f"🎉 恭喜你猜中了！答案是 {state['answer']}。\n"
                        f"你總共猜了 {state['attempts']} 次！\n"
                        "輸入『開始遊戲』可以再玩一次。"
                    )
                    state["active"] = False

            except ValueError:
                reply = "請輸入有效的整數數字！"


        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply)]
            )
        )


app.run()


# 展望

猜錯次數排行榜（用 Firebase 或簡單 CSV 儲存）

難度調整（例如猜 1～1000）

加入「放棄遊戲」指令

自動記錄遊戲結果並回顧歷史成績